In [1]:
print("Hello World")

Hello World


In [ ]:
import os
from langchain_community.graphs import Neo4jGraph
from langchain_groq import ChatGroq
from langchain_core.documents import Document
from langchain_experimental.graph_transformers import LLMGraphTransformer
from langchain.chains import GraphCypherQAChain

In [ ]:
load_dotenv(".env")
api_key = os.getenv("API_KEY")
NEO4J_URI = os.getenv("NEO4J_URI")
NEO4J_USERNAME = os.getenv("NEO4J_USER")
NEO4J_PASSWORD = os.getenv("NEO4J_PASSWORD")

In [ ]:
os.environ["NEO4J_URI"]=NEO4J_URI
os.environ["NEO4J_USERNAME"]=NEO4J_USERNAME
os.environ["NEO4J_PASSWORD"]=NEO4J_PASSWORD

## **Setup Neo4j**

In [ ]:
graph=Neo4jGraph(
    url=NEO4J_URI,
    username=NEO4J_USERNAME,
    password=NEO4J_PASSWORD,
)

## **Load LLM from Groq**

In [ ]:
llm=ChatGroq(groq_api_key=groq_api_key,model_name="llama-3.1-8b-instant")

## **Insert data into GraphDB**

In [ ]:
text="""
Soekarno (born June 6, 1901 – June 21, 1970) was the first President of Indonesia, serving from 1945 to 1967. 
He played a pivotal role in the country's struggle for independence from the Netherlands and is considered one 
of the most important figures in Indonesian history. Soekarno was a charismatic orator and leader, known for 
his vision of a united Indonesia based on the principles of Pancasila, which he introduced as the ideological 
foundation of the state.

Born in Surabaya, Soekarno was educated in Dutch schools and later attended the Technische Hoogeschool 
(now Bandung Institute of Technology), where he studied civil engineering. During his studies, he became 
active in nationalist movements and began advocating for Indonesian independence.

In 1945, Soekarno, along with Mohammad Hatta, proclaimed Indonesia's independence. As president, he 
sought to unify the diverse archipelago under a single government. However, his presidency faced numerous 
challenges, including political instability, economic difficulties, and regional rebellions. In 1965, a coup 
attempt led to significant changes in Indonesia's political landscape, eventually resulting in Soekarno's 
departure from power and the rise of Suharto.

Despite his controversial later years, Soekarno remains a revered figure in Indonesia, celebrated as the 
"Proclaimer of Independence" and a symbol of national unity and resilience.
 """
documents=[Document(page_content=text)]
documents

In [ ]:
llm_transformer=LLMGraphTransformer(llm=llm)

In [ ]:
graph_documents=llm_transformer.convert_to_graph_documents(documents)

In [ ]:
# graph_documents
graph_documents[0].nodes
#graph_documents[0].relationships

In [ ]:
for doc in graph_documents:
    # Insert nodes
    for node in doc.nodes:
        query = f"""
        MERGE (n:{node['type']} {{id: $id}})
        SET n += $properties
        """
        graph.run_query(query, parameters={"id": node['id'], "properties": node['properties']})

    # Insert relationships
    for relationship in doc.relationships:
        query = f"""
        MATCH (a {{id: $start_node_id}})
        MATCH (b {{id: $end_node_id}})
        MERGE (a)-[r:{relationship['type']}]->(b)
        SET r += $properties
        """
        graph.run_query(
            query,
            parameters={
                "start_node_id": relationship['source']['id'],
                "end_node_id": relationship['target']['id'],
                "properties": relationship['properties'],
            },
        )

## **GraphRAG using Langchain**

In [ ]:
chain=GraphCypherQAChain.from_llm(llm=llm,
                                 graph=graph,
                                 verbose=True,
                                 top_k=5,
                                 allow_dangerous_requests=True # Add this line to acknowledge the risk
                                )

In [ ]:
response=chain.invoke({"query":"Who is Soekarno?"})

response